In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import json
from collections import namedtuple
from datetime import datetime
from kafka import KafkaProducer
import time

 
f = open('hashtags.json')
hashtags = json.load(f)
f.close()

hash_values = ["#"+i.lower() for i in list(hashtags.values())]
hash_values

['#twittersold', '#rentalhealthday', '#elonmusk', '#kartikaaryan', '#suga']

In [2]:
sc = SparkContext("local[2]", "dbt-assignment")

22/04/26 14:57:12 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 10.5.26.32 instead (on interface wlo1)
22/04/26 14:57:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/26 14:57:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
ssc = StreamingContext(sc, 1 )
sqlContext = SQLContext(sc)

/home/sumukhbhat2701/.local/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
lines = ssc.socketTextStream("127.0.0.1", 6002).window(300, 300)


In [5]:
fields = ("Hashtag", "Count")
Tweet = namedtuple( 'Tweet', fields )


In [6]:
def myRead(rdd):
    if not rdd.isEmpty():
        return rdd.toDF().sort("Count").createOrReplaceTempView("tweets")

processed = ( lines.flatMap( lambda text: text.split( " " ) )
  .filter( lambda word: word.lower().startswith("#") and word.lower() in hash_values)
  .map( lambda word: ( word.lower(), 1 ) )
  .reduceByKey( lambda a, b: a + b ) 
  .map( lambda rec: Tweet( rec[0], rec[1] ) )) 

processed.pprint()

processed.foreachRDD( lambda rdd: myRead(rdd) ) # Registers to a table.

In [7]:
def serializer(message):
    return json.dumps(message).encode('utf-8')


# Kafka Producer
producer = KafkaProducer(
    bootstrap_servers=['localhost:9092'],
    value_serializer=serializer
)

def send_to_kafka(m):
    mydate = datetime.now()
    for i in m:
        print("producing: " + i + ":"+ str(m[i]))
        producer.send(i.split('#')[1], i+";"+str(m[i])+";"+str(mydate))

In [8]:
ssc.start()


22/04/26 14:57:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:16 WARN BlockManager: Block input-0-1650965236000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:16 WARN BlockManager: Block input-0-1650965236200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:16 WARN BlockManager: Block input-0-1650965236400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:16 WARN BlockManager: Block input-0-1650965236600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:17 WARN BlockManager: Block input-0-1650965237000 replicated to

In [ ]:
count = 0
x = []
while count < 6:
    d = {}
    time.sleep(310)
    tweets = sqlContext.sql( 'select Hashtag, Count from tweets' )
    df = tweets.toPandas()
    d1 = df.set_index('Hashtag').T.to_dict('list')
    for i in hash_values:
        if i not in d1:
            d[i] = 0
        else:
            d[i] = d1[i][0]
    send_to_kafka(d)
    count = count + 1
    x.append(d1)

22/04/26 14:57:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:22 WARN BlockManager: Block input-0-1650965242200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:22 WARN BlockManager: Block input-0-1650965242400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:22 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:22 WARN BlockManager: Block input-0-1650965242600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:23 WARN BlockManager: Block input-0-1650965242800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:23 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:23 WARN BlockManager: Block input-0-1650965243000 replicated to

22/04/26 14:57:30 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:30 WARN BlockManager: Block input-0-1650965250400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:30 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:30 WARN BlockManager: Block input-0-1650965250600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:31 WARN BlockManager: Block input-0-1650965250800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:31 WARN BlockManager: Block input-0-1650965251000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:31 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:31 WARN BlockManager: Block input-0-1650965251200 replicated to

22/04/26 14:57:38 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:38 WARN BlockManager: Block input-0-1650965258600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:39 WARN BlockManager: Block input-0-1650965258800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:39 WARN BlockManager: Block input-0-1650965259000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:39 WARN BlockManager: Block input-0-1650965259200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:39 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:39 WARN BlockManager: Block input-0-1650965259400 replicated to

22/04/26 14:57:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:47 WARN BlockManager: Block input-0-1650965266800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:47 WARN BlockManager: Block input-0-1650965267000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:47 WARN BlockManager: Block input-0-1650965267200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:47 WARN BlockManager: Block input-0-1650965267400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:47 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:47 WARN BlockManager: Block input-0-1650965267600 replicated to

22/04/26 14:57:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:55 WARN BlockManager: Block input-0-1650965275000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:55 WARN BlockManager: Block input-0-1650965275200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:55 WARN BlockManager: Block input-0-1650965275400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:55 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:55 WARN BlockManager: Block input-0-1650965275600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 14:57:56 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:57:56 WARN BlockManager: Block input-0-1650965275800 replicated to

...

22/04/26 14:59:59 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 14:59:59 WARN BlockManager: Block input-0-1650965399600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:00 WARN BlockManager: Block input-0-1650965400000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:00 WARN BlockManager: Block input-0-1650965400200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:00 WARN BlockManager: Block input-0-1650965400400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:00 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:00 WARN BlockManager: Block input-0-1650965400600 replicated to

22/04/26 15:00:08 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:08 WARN BlockManager: Block input-0-1650965408200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:08 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:08 WARN BlockManager: Block input-0-1650965408400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:08 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:08 WARN BlockManager: Block input-0-1650965408600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:09 WARN BlockManager: Block input-0-1650965408800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:09 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:09 WARN BlockManager: Block input-0-1650965409000 replicated to

22/04/26 15:00:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:16 WARN BlockManager: Block input-0-1650965416400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:16 WARN BlockManager: Block input-0-1650965416600 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:17 WARN BlockManager: Block input-0-1650965417000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:17 WARN BlockManager: Block input-0-1650965417200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:17 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:17 WARN BlockManager: Block input-0-1650965417400 replicated to

22/04/26 15:00:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:25 WARN BlockManager: Block input-0-1650965424800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:25 WARN BlockManager: Block input-0-1650965425000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:25 WARN BlockManager: Block input-0-1650965425200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:25 WARN BlockManager: Block input-0-1650965425400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:25 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:25 WARN BlockManager: Block input-0-1650965425600 replicated to

22/04/26 15:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:33 WARN BlockManager: Block input-0-1650965432800 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:33 WARN BlockManager: Block input-0-1650965433000 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:33 WARN BlockManager: Block input-0-1650965433200 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:33 WARN BlockManager: Block input-0-1650965433400 replicated to only 0 peer(s) instead of 1 peers
22/04/26 15:00:33 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
22/04/26 15:00:33 WARN BlockManager: Block input-0-1650965433600 replicated to

In [11]:
ssc.stop()

22/04/26 01:28:48 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.net.SocketInputStream.socketRead0(Native Method)
	at java.net.SocketInputStream.socketRead(SocketInputStream.java:116)
	at java.net.SocketInputStream.read(SocketInputStream.java:171)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.io.InputStreamReader.read(InputStreamReader.java:184)
	at java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.io.BufferedReader.readLine(BufferedReader.java:324)
	at java.io.BufferedReader.readLine(BufferedReader.java:389)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:119)
	at org.apache.spar